<a href="https://colab.research.google.com/github/Hsuan7/homework/blob/main/w11_%E5%A4%9A%E6%A8%A1%E6%85%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM, Concatenate, Input, Dropout
from keras.models import Model

# 1. 載入並預處理股價數據
stock_data = pd.read_csv('/content/drive/MyDrive/week11_ Multimodal/kaggle/DJIA_table(train).csv', index_col='Date', parse_dates=True)  # 讀取股價數據，並將日期作為索引
scaler = MinMaxScaler(feature_range=(0, 1))  # 初始化MinMaxScaler，將數據縮放到0到1之間
scaled_stock_data = scaler.fit_transform(stock_data[['Close']])  # 對收盤價數據進行縮放處理

<ipython-input-6-24b65902d48f>:12: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  stock_data = pd.read_csv('/content/drive/MyDrive/week11_ Multimodal/kaggle/DJIA_table(train).csv', index_col='Date', parse_dates=True)  # 讀取股價數據，並將日期作為索引


In [7]:


# 2. 載入並預處理情緒數據
sentiment_data = pd.read_csv('/content/drive/MyDrive/week11_ Multimodal/kaggle/RedditNews(train).csv', index_col='Date', parse_dates=True)  # 讀取新聞情緒數據，並將日期作為索引
sentiment_data.fillna('', inplace=True)  # 將空值填充為空字符串

# 將每日的所有新聞標題合併為一個字符串
sentiment_data_grouped = sentiment_data.groupby(sentiment_data.index).agg({'News': ' '.join})  # 按日期合併新聞標題

# 使用簡單的情緒評分（例如詞彙數量）作為情緒分析的占位符
sentiment_data_grouped['Sentiment_Score'] = sentiment_data_grouped['News'].apply(lambda x: len(x.split()))  # 計算每行的單詞數作為情緒得分
scaled_sentiment_data = scaler.fit_transform(sentiment_data_grouped[['Sentiment_Score']])  # 對情緒得分進行縮放處理



In [8]:
# 3. 準備訓練和測試數據集
def create_dataset(data, look_back=1):
    X, Y = [], []
    for i in range(len(data) - look_back - 1):
        X.append(data[i:(i + look_back), :])  # 將look_back個時間步的數據作為輸入
        Y.append(data[i + look_back, 0])  # 將下一個時間步的目標值作為輸出
    return np.array(X), np.array(Y)

look_back = 5  # 設定回溯時間步數

# 股價數據集
X_stock, Y_stock = create_dataset(scaled_stock_data, look_back)  # 創建股價數據的輸入和輸出

# 情緒數據集
X_sentiment, Y_sentiment = create_dataset(scaled_sentiment_data, look_back)  # 創建情緒數據的輸入和輸出

# 確保兩個數據集具有相同的長度
min_length = min(len(X_stock), len(X_sentiment))
X_stock, Y_stock = X_stock[:min_length], Y_stock[:min_length]
X_sentiment, Y_sentiment = X_sentiment[:min_length], Y_sentiment[:min_length]

# 將數據分為訓練和測試數據
X_stock_train, X_stock_test, Y_train, Y_test = train_test_split(X_stock, Y_stock, test_size=0.2, random_state=42)  # 分割股價數據集
X_sentiment_train, X_sentiment_test, _, _ = train_test_split(X_sentiment, Y_sentiment, test_size=0.2, random_state=42)  # 分割情緒數據集

In [9]:
# 4. 定義股價的LSTM模型
input_stock = Input(shape=(look_back, X_stock.shape[2]))  # 定義股價數據的輸入層
stock_lstm = LSTM(50, return_sequences=False)(input_stock)  # 添加LSTM層
stock_dense = Dense(50, activation='relu')(stock_lstm)  # 添加全連接層

# 5. 定義情緒數據的LSTM模型
input_sentiment = Input(shape=(look_back, X_sentiment.shape[2]))  # 定義情緒數據的輸入層
sentiment_lstm = LSTM(50, return_sequences=False)(input_sentiment)  # 添加LSTM層
sentiment_dense = Dense(50, activation='relu')(sentiment_lstm)  # 添加全連接層

In [10]:
# 6. 合併兩個模型的輸出
merged = Concatenate()([stock_dense, sentiment_dense])  # 將股價和情緒模型的輸出合併
merged_dense = Dense(50, activation='relu')(merged)  # 添加全連接層
dropout = Dropout(0.2)(merged_dense)  # 添加Dropout層防止過擬合
output = Dense(1)(dropout)  # 添加輸出層

In [13]:

# 7. 編譯並訓練模型
model = Model(inputs=[input_stock, input_sentiment], outputs=output)  # 定義模型
model.compile(optimizer='adam', loss='mean_squared_error')  # 編譯模型，使用均方誤差作為損失函數
model.fit([X_stock_train, X_sentiment_train], Y_train, epochs=20, batch_size=32, validation_data=([X_stock_test, X_sentiment_test], Y_test))  # 訓練模型

# 8. 進行預測
y_pred = model.predict([X_stock_test, X_sentiment_test])  # 使用測試數據進行預測




Epoch 1/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.0020 - val_loss: 2.6372e-04
Epoch 2/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0020 - val_loss: 3.4070e-04
Epoch 3/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0019 - val_loss: 3.2230e-04
Epoch 4/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0020 - val_loss: 4.5667e-04
Epoch 5/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0019 - val_loss: 5.8836e-04
Epoch 6/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0018 - val_loss: 0.0010
Epoch 7/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0020 - val_loss: 2.9636e-04
Epoch 8/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0021 - val_loss: 2.8383e-04
Epoch 9/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0015 - val_loss: 5.4703e-04
Epoch 10/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0019 - val_loss: 6.4674e-04
Epoch 11/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0017 - val_loss: 2.3758e-04
Epoch 12/20
47/47 ━━━━━━━━━━━━━━━

In [16]:
# 9. 評估模型
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(Y_test, y_pred)  # 計算均方誤差
r2 = r2_score(Y_test, y_pred)  # 計算R^2分數作為模型的準確率
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score (Accuracy): {r2}')

Mean Squared Error: 0.00020947629628074557
R^2 Score (Accuracy): 0.9970755825334396
